In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import torch
from Constants import *
from Preprocessing import *
from Models import *
import copy
from Utils import *
from SymptomPrediction import *

pd.set_option('display.max_rows', 200)



In [3]:
data = DTDataset(use_smote=False)
data.processed_df.T
data.get_input_state(1).shape
# data.processed_df#.shape, len(data.processed_df.index.unique())

(536, 61)

In [23]:
for c in data.processed_df.columns:
    print(c)

hpv
age
packs_per_year
gender
Aspiration rate Pre-therapy
total_dose
dose_fraction
OS (Calculated)
Locoregional control (Time)
FDM (months)
time_to_event
Overall Survival (1=alive, 0=dead)
LRC
DC
bilateral
White/Caucasion
Hispanic/Latino
African American/Black
Asian
cc_none
cc_platinum
cc_cetuximab
cc_others
no_dose_adjustment
dose_modified
dose_delayed
dose_cancelled
dose_delayed_&_modified
regiment_modification
T-category_1
T-category_2
T-category_3
T-category_4
N-category_0
N-category_1
N-category_2
N-category_3
AJCC_1
AJCC_2
AJCC_3
AJCC_4
Pathological Grade_0
Pathological Grade_1
Pathological Grade_2
Pathological Grade_3
Pathological Grade_4
subsite_BOT
subsite_GPS
subsite_NOS
subsite_Soft palate
subsite_Tonsil
treatment_CC
treatment_IC+CC
treatment_IC+Radiation alone
treatment_Radiation alone
DLT_Dermatological
DLT_Neurological
DLT_Gastrointestinal
DLT_Hematological
DLT_Nephrological
DLT_Vascular
DLT_Infection (Pneumonia)
DLT_Other
DLT_Dermatological 2
DLT_Neurological 2
DLT_Gastr

In [5]:
def load_mdasi_stuff():
    model = torch.load('../resources/symptomImputer.pt')
    mdasi_base = pd.read_excel('../data/mdasi_updated.xlsx').drop('Unnamed: 0',axis=1)
    mdasi_imputed = pd.read_csv('../data/mdasi_imputed.csv').drop('Unnamed: 0',axis=1)
    mdasi_base['id'] = mdasi_base['ID'].apply(lambda x: int(x.replace('STIEFEL_','')))
    mdasi_imputed['id'] = mdasi_imputed['ID'].apply(lambda x: int(x.replace('STIEFEL_','')))
    return model, mdasi_base, mdasi_imputed

sp, mdasib, mdasi = load_mdasi_stuff()
mdasi.shape, mdasib.shape

((937, 234), (1192, 295))

In [6]:
process_mdasi_input(mdasi).mean()

age                                   60.514408
gender                                 0.906083
packs_per_year                         7.441222
hpv                                    1.000000
total_dose                          6838.928495
dose_fraction                         32.559232
White/Caucasion                        0.927428
Hispanic/Latino                        0.067236
African American/Black                 0.025614
Asian                                  0.006403
bilateral                              0.022412
subsite_BOT                            0.465315
subsite_GPS                            0.011740
subsite_Tonsil                         0.454642
subsite_Soft palate                    0.005336
subsite_NOS                            0.059765
T-category_1                           0.326574
N-category_0                           0.073639
T-category_2                           0.356457
N-category_1                           0.671291
T-category_3                           0

In [1]:
mdasi

NameError: name 'mdasi' is not defined

In [89]:
def nan_mse_loss(ypred, y):
    #ignores loss in the autoencoder for missing values (-1 here)
    y = torch.flatten(y)
    ypred = torch.flatten(ypred)
    mask = torch.lt(y,-.1)
    out = (ypred[~mask] - y[~mask])**2
    loss = out.mean()
    return loss

def train_symptom_model(mdasi,lr=.0001,patience=1000,epochs=1000000,symptoms = None,
                        save_file='../resources/symptomImputerTemp.pt',verbose=True,**kwargs):

    mdasi_input=  process_mdasi_input(mdasi)

    train_ids = mdasi_input.reset_index().id.sample(frac=.66,replace=False).values
    test_ids = mdasi_input.drop(train_ids).reset_index().id.values
    
    xtrain = df_to_torch(mdasi_input.loc[train_ids])
    xtest = df_to_torch(mdasi_input.loc[test_ids])

    sdf,_,symptoms = get_symptom_df(mdasi)

    stopred= symptoms
    if symptoms is None:
        stopred = Const.prediction_symptoms

    ytrain = torch.from_numpy(sdf_symptom_array( sdf.loc[train_ids], stopred))
    ytest = torch.from_numpy(sdf_symptom_array( sdf.loc[test_ids], stopred))

    sp = SymptomPredictor(xtrain.shape[1],ytrain.shape[1],max_rating=10,**kwargs)
    sp.fit_normalizer(xtrain)
    optimizer = torch.optim.Adam(sp.parameters(),lr=lr)

    best_loss = 1000000000000000000000000
    steps_since_improvement=0
    patience=patience
    sp.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        ypred = sp(xtrain)

        loss = nan_mse_loss(ypred,ytrain)
        loss.backward()
        optimizer.step()
        ypred_test = sp(xtest)
        val_loss = nan_mse_loss(ypred_test,ytest)
        if verbose:
            print(epoch,loss.item(),val_loss.item())
        steps_since_improvement+=1
        if val_loss.item() < best_loss:
            best_loss = val_loss.item()
            steps_since_improvement=0
            torch.save(sp.state_dict(),save_file)
        if steps_since_improvement > patience:
            break
    sp.load_state_dict(torch.load(save_file))
    sp.eval()
    if verbose:
        print('best loss',best_loss,epoch-patience)
    return sp,best_loss

# sp,sp_loss = train_symptom_model(mdasib,hidden_layers=[1000,100,10])
# sp

In [91]:
hidden_layersets = [
    [10],[100],[1000,100],[5000,100],
    [10,10],[100,100],
    [1000,50],[5000,50]
]
dropouts = [0,.1,.2,.5]
best_model = None
best_mloss = 1000000000000000000
results = []
for hl in hidden_layersets:
    for do in dropouts:
        sptemp,sploss = train_symptom_model(mdasi,hidden_layers=hl,dropout=do,patience=100,verbose=False)
        results.append({'model': sptemp, 'loss': sploss,'layers': hl[:],'dropout':do})
        print()
        print(results[-1])
        if sploss < best_mloss:
            best_model = sptemp
            best_mloss = sploss
            print('+++++++++++++++++++++++++++')
            print('new best loss',sploss)
            print(hl)
            print(do)
            print('_________++++++++++__________')


{'model': SymptomPredictor(
  (layers): ModuleList(
    (0): Linear(in_features=27, out_features=10, bias=True)
    (1): ReLU()
  )
  (batchnorm): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0, inplace=False)
  (relu): ReLU()
  (sigmoid): Sigmoid()
  (final_layer): Linear(in_features=10, out_features=112, bias=True)
), 'loss': 3.572778029530631, 'layers': [10], 'dropout': 0}
+++++++++++++++++++++++++++
new best loss 3.572778029530631
[10]
0
_________++++++++++__________

{'model': SymptomPredictor(
  (layers): ModuleList(
    (0): Linear(in_features=27, out_features=10, bias=True)
    (1): ReLU()
  )
  (batchnorm): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (relu): ReLU()
  (sigmoid): Sigmoid()
  (final_layer): Linear(in_features=10, out_features=112, bias=True)
), 'loss': 5.310018261393623, 'layers': [10], 'dropout': 0.1}

{'model': SymptomPredi


{'model': SymptomPredictor(
  (layers): ModuleList(
    (0): Linear(in_features=27, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=10, bias=True)
    (3): ReLU()
  )
  (batchnorm): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (relu): ReLU()
  (sigmoid): Sigmoid()
  (final_layer): Linear(in_features=10, out_features=112, bias=True)
), 'loss': 5.43094330054629, 'layers': [10, 10], 'dropout': 0.1}

{'model': SymptomPredictor(
  (layers): ModuleList(
    (0): Linear(in_features=27, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=10, bias=True)
    (3): ReLU()
  )
  (batchnorm): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (relu): ReLU()
  (sigmoid): Sigmoid()
  (final_layer): Linear(in_features=10, out_features=112, bias=True)
), 'loss': 4.9487019947

In [92]:
best_model

SymptomPredictor(
  (layers): ModuleList(
    (0): Linear(in_features=27, out_features=10, bias=True)
    (1): ReLU()
  )
  (batchnorm): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0, inplace=False)
  (relu): ReLU()
  (sigmoid): Sigmoid()
  (final_layer): Linear(in_features=10, out_features=112, bias=True)
)

In [93]:
torch.save(best_model,'../resources/symptomImputer.pt')

In [100]:
mdasi_embeddings = best_model.get_embedding(df_to_torch(process_mdasi_input(mdasi))).cpu().detach()
pd.DataFrame(mdasi_embeddings.cpu().detach().numpy(),index=mdasi.id).to_csv('../resources/mdasi_embeddings.csv')
pd.read_csv('../resources/mdasi_embeddings.csv')

,id,0,1,2,3,4,5,6,7,8,9
0,1445,0.500248,1.414331,0.547527,-0.238426,-1.361126,-1.260582,1.397747,0.993124,-1.137796,2.373032
1,1123,1.487083,1.007057,-0.035258,1.829207,-1.361126,-0.713692,-0.538427,-0.318023,-1.137796,0.779958
2,1446,0.812521,0.903847,2.806787,-0.877378,-1.361126,-1.260582,2.524730,1.340965,-1.137796,-0.422416
3,1448,0.806867,0.387447,1.268020,1.322180,-1.361126,-1.260582,1.342638,-0.318023,-1.137796,1.311435
4,1449,1.225510,0.541784,0.852631,1.462089,-0.814546,-1.260582,0.769261,-0.318023,-0.824981,0.200049
...,...,...,...,...,...,...,...,...,...,...,...
932,1707,1.103811,1.428927,0.264068,0.756452,-0.168224,-1.260582,0.234492,2.886616,-1.137796,1.772260
933,1708,1.696921,0.924384,0.906366,1.670223,-0.942027,-1.260582,-0.243785,0.443138,-1.018400,-0.060345
934,1709,1.878426,1.065306,-0.527575,1.261594,-1.361126,-1.260582,-1.135325,2.643781,-1.137796,0.380769
935,1710,1.450276,1.032895,0.962017,1.134421,-0.537424,-1.260582,0.238278,0.804344,-1.137796,0.421281


In [101]:
def get_test_patient(d,pid=7,clear_transitions=True):
    tp = d.processed_df.loc[pid].to_dict()
    if clear_transitions:
        for v in Const.primary_disease_states + Const.nodal_disease_states:
            tp[v] = 0
            tp[v + ' 2'] = 0
    return tp 

def get_knn(model,xin,mdf,sdf,symptom_subset=None,k=10,dates=[0,7,12,27]):
    xalt = df_to_torch(mdf)
    embeddings = model.get_embedding(xin).cpu().detach().numpy()
    base_embeddings = model.get_embedding(xalt).cpu().detach().numpy()
    dists = cdist(embeddings,base_embeddings)[0]
    order = np.argsort(dists)[:k]
    dists = dists[order]
    
    mdasi_ids= mdf.index
    ids = mdasi_ids[order]
    symptoms = sdf.loc[ids]
    
    if symptom_subset is None:
        symptom_subset = Const.prediction_symptoms
    res = {'ids': ids.tolist(),'dists': dists.tolist()}
    sentries = {}
    for sym in symptom_subset:
        if sym == 'core':
            continue
        cols = [c for c in symptoms.columns if sym+'_' in c]
        values = symptoms[cols].values
        entry = {'ratings': values.tolist()}
        means = []
        for cidx in range(values.shape[1]):
            subvals = values[:,cidx]
            subvals = [v for v in subvals if v >= 0]
            means.append(np.mean(subvals))
        entry['means'] = means
        sentries[sym] = entry
    return {'ids': ids.tolist(),'dists':dists.tolist(),'symptoms':sentries}

def get_knn_predictions(fdict,
                        model,
                        mdasi,
                        mdasi_embeddings=None,
                        k=10,
                        ttype=torch.FloatTensor,
                        dates=[0,7,12,27],
                        symptom_subset = None,
                        decision_state=0,#0 is default, 1,2,3 are ic,cc,nd?
                       ):
    

    mdasi_df = process_mdasi_input(mdasi)
    sdf,output_dates,output_symptoms = get_symptom_df(mdasi)
    
    
    order = mdasi_df.columns
    xin = torch.tensor([fdict[k] for k in order]).type(ttype).view(1,-1) 

    if decision_state == 0:
        se = get_knn(model,xin,mdasi_df,sdf,k=k,symptom_subset=symptom_subset,dates=dates)
        se['dates'] = dates
        res = se
    
    else:
        dcol = Const.decisions[decision_state-1]
        print(dcol)
        mdasi_df_treated = mdasi_df[mdasi_df[dcol].astype(float) > .5]
        mdasi_df_untreated = mdasi_df[mdasi_df[dcol].astype(float) < .5]
        se_treated = get_knn(model,xin,mdasi_df_treated,sdf,k=k,symptom_subset=symptom_subset,dates=dates)
        se_untreated = get_knn(model,xin,mdasi_df_untreated,sdf,k=k,symptom_subset=symptom_subset,dates=dates)
        res = {'treated': se_treated,'untreated': se_untreated,'dates': dates}
    return res

test_patient = get_test_patient(data,7)
get_knn_predictions(test_patient,sp2,mdasi,symptom_subset=['choke','drymouth'],decision_state=1)

Decision 1 (Induction Chemo) Y/N


{'treated': {'ids': [1069, 118, 225, 1368, 1605, 164, 1342, 1015, 1542, 1532],
  'dists': [40.595754742170485,
   40.99183677704289,
   41.256458873501465,
   41.35526375542134,
   41.452049965318615,
   41.52398205700639,
   41.564073532250795,
   41.78066071929792,
   41.78761053841052,
   41.8526227330986],
  'symptoms': {'choke': {'ratings': [[0.0, 0.0, 1.0, 0.0],
     [0.0, 2.0, 0.758286357, 2.0],
     [0.0, 1.0, 0.0, 0.0],
     [0.641221374, 2.0, 0.0, 0.0],
     [1.0, 0.0, 0.453594029, 0.0],
     [0.0, 2.142912626, 1.0, 3.0],
     [0.0, 0.0, 0.0, 0.0],
     [0.0, 0.0, 0.376850367, 0.0],
     [0.0, 2.0, 0.766977549, 1.115396976],
     [0.641221374, 0.0, 0.450264215, 0.536481261]],
    'means': [0.22824427480000004,
     0.9142912626000002,
     0.4805972517,
     0.6651878237]},
   'drymouth': {'ratings': [[0.0, 0.0, 1.0, 4.0],
     [0.0, 1.0, 1.703879237, 6.0],
     [0.0, 1.0, 5.0, 6.0],
     [0.945638432, 4.0, 3.0, 2.0],
     [0.0, 6.0, 4.932738304, 5.0],
     [4.0, 6.488459587,

In [102]:
def get_symptom_predictions(fdict,
                        model,
                        order=None,
                        k=10,
                        ttype=torch.FloatTensor,
                        dates=[0,7,12,27],
                        symptom_subset = None,
                        decision_state=0,#0 is default, 1,2,3 are ic,cc,nd?
                       ):

    
    if order is None:
        order = Const.mdasi_input_cols

    xin = torch.tensor([fdict[k] for k in order]).type(ttype).view(1,-1) 

    model.eval()
    model.disable_dropout()
    ypred = model(xin).cpu().detach().numpy()

    symptoms = Const.prediction_symptoms
    if symptom_subset is None:
        symptom_subset = Const.prediction_symptoms
    curpos = 0
    results = {s: {'means':[],'ratings': []} for s in symptom_subset}
    
    for sym in symptoms:
        if sym not in symptom_subset:
            curpos += len(dates)
            continue
        values = ypred[:,curpos:curpos+len(dates)].tolist()[0]
        curpos += len(dates)
        results[sym]['means'] = values
        
    for i in range(k):
        model.enable_dropout()
        ypred = model(xin).cpu().detach().numpy()
        curpos = 0
        for sym in symptoms:
            if sym not in symptom_subset:
                curpos += len(dates)
                continue
            values = ypred[:,curpos:curpos+len(dates)].tolist()[0]
            curpos += len(dates)
            cur_res = results[sym]['ratings']
            cur_res.append(values)
            results[sym]['ratings'] = cur_res
    model.disable_dropout()
    return {'dates': dates,'symptoms': results}

test_patient = get_test_patient(data,5)
get_symptom_predictions(test_patient,sp,symptom_subset=['choke','drymouth'])

{'dates': [0, 7, 12, 27],
 'symptoms': {'choke': {'means': [6.100060939788818, 10.0, 10.0, 10.0],
   'ratings': [[8.176459312438965, 10.0, 10.0, 10.0],
    [2.87192702293396, 10.0, 10.0, 10.0],
    [2.1255526542663574,
     4.624366283416748,
     1.6369287967681885,
     1.4604820013046265],
    [8.405499458312988, 10.0, 10.0, 10.0],
    [2.015972375869751,
     5.311256408691406,
     2.0509910583496094,
     1.7925984859466553],
    [1.6517019271850586,
     4.735230922698975,
     2.233750104904175,
     2.3451006412506104],
    [8.31027889251709,
     9.208638191223145,
     6.922463417053223,
     9.567170143127441],
    [2.952191114425659, 10.0, 10.0, 10.0],
    [8.343428611755371, 10.0, 10.0, 10.0],
    [9.011286735534668, 10.0, 10.0, 10.0]]},
  'drymouth': {'means': [4.6650214195251465,
    7.582496166229248,
    5.747006416320801,
    8.741512298583984],
   'ratings': [[5.53472375869751, 10.0, 8.317580223083496, 10.0],
    [2.568161964416504,
     4.523853778839111,
     4.53

In [12]:
DOSE_ORDER = ['Esophagus', 'Spinal_Cord', 'Lt_Brachial_Plexus', 'Rt_Brachial_Plexus', 'Cricopharyngeal_Muscle', 'Cricoid_cartilage', 'IPC', 'MPC', 'Brainstem', 'Larynx', 'Thyroid_cartilage', 'Rt_Sternocleidomastoid_M', 'Rt_Mastoid', 'Rt_Parotid_Gland', 'Rt_Medial_Pterygoid_M', 'Rt_Lateral_Pterygoid_M', 'Rt_Masseter_M', 'Lt_Sternocleidomastoid_M', 'Lt_Mastoid', 'Lt_Parotid_Gland', 'Lt_Submandibular_Gland', 'Lt_Medial_Pterygoid_M', 'Lt_Lateral_Pterygoid_M', 'Lt_Masseter_M', 'Supraglottic_Larynx', 'SPC', 'Rt_Submandibular_Gland', 'Hyoid_bone', 'Soft_Palate', 'Genioglossus_M', 'Tongue', 'Rt_Ant_Digastric_M', 'Lt_Ant_Digastric_M', 'Mylogeniohyoid_M', 'Extended_Oral_Cavity', 'Mandible', 'Hard_Palate', 'Lower_Lip', 'Upper_Lip', 'Glottic_Area']
dosedf = pd.read_csv('../data/minimal_dose_cluster_dataset.csv')
for c in dosedf.columns:
    print(c)

id
D10
D15
D2
D20
D25
D30
D35
D40
D45
D5
D50
D55
D60
D65
D70
D75
D80
D85
D90
D95
D97
D98
D99
V10
V15
V20
V25
V30
V35
V40
V45
V5
V50
V55
V60
V65
V70
V75
V80
level_0
max_dose
mean_dose
min_dose
volume
chemotherapy
baseline_height
sx_prior_to_enrollment
ic_prior_to_enrollment
status_at_enrollememt
end_of_treatment_weight
rt
baseline_weight
is_ajcc_8th_edition
is_male
age
n_stage
performance_score
concurrent_prior_to_enrollment
ic
concurrent
sxprimary
t_stage
os
subsite
M6_mbs_digest
Local_control
wk6_weight
hpv
Technique
Regional_control
dates
symptoms_pain_original
symptoms_fatigue_original
symptoms_nausea_original
symptoms_sleep_original
symptoms_distress_original
symptoms_sob_original
symptoms_memory_original
symptoms_appetite_original
symptoms_drowsy_original
symptoms_drymouth_original
symptoms_sad_original
symptoms_vomit_original
symptoms_numb_original
symptoms_mucus_original
symptoms_swallow_original
symptoms_choke_original
symptoms_voice_original
symptoms_skin_original
symptoms_con

In [32]:
def get_camprt_data():
    pdata = pd.read_csv('../data/camprt_pdata.csv')
#     for col in pdata.columns:
#         print(col)
#         if len(pdata[col].unique()) < 10:
#             print(pdata[col].unique())
#         print('___')

    to_keep = ['id','age','gender','bilateral','total_dose','dose_fraction','hpv','packs_per_year']
    cdf = pdata.copy()
    cdf['id'] = pdata['Dummy ID']
    cdf['age'] = pdata['Age at Diagnosis (Calculated)']
    cdf['gender'] = pdata['Gender'].apply(lambda x: x=='Male')
    cdf['bilateral'] = pdata['Tm Laterality (R/L)'].apply(lambda x: x.lower() == 'bilateral')
    cdf['total_dose'] = pdata['Total dose']
    cdf['dose_fraction'] = pdata['Dose/fraction (Gy)']
    cdf['packs_per_year'] = pdata['Smoking status (Packs/Year)']
    
    cdf[Const.decisions[0]] = pdata['Therapeutic combination'].apply(lambda x: 'IC' in x)
    cdf[Const.decisions[1]] = pdata['Therapeutic combination'].apply(lambda x: 'CC' in x)
    smoking_map = {
        
    }
    hpv_map = {
        'Unknown': -1,
        'Negative': 0,
        'Positive': 1
    }
    cdf['hpv'] = pdata['HPV/P16 status'].apply(lambda x: hpv_map.get(x,-1))
    
    tstages = [1,2,3,4]

    ajcc_map = {
        'II': 2,
        'I': 1,
        'III': 3,
        'IV': 4,
    }
    
    path_grade_map = {
        'I': 1,
        'II': 2,
        'III': 3,
        'IV': 4
    }
    for ts in tstages:
        cdf['T-category_'+ str(ts)] = pdata['T-category'].apply(lambda x: int(x[1]) == ts if x.lower() not in ['tx','tis'] else False)
        to_keep.append('T-category_'+str(ts))

        ns = ts-1
        cdf['N-category_'+ str(ns)] = pdata['N-category_8th_edition'].apply(lambda x: int(x[1]) == ns)
        to_keep.append('N-category_'+str(ns))

        cdf['AJCC_' + str(ts)] = pdata['AJCC 8th edition'].apply(lambda x: ajcc_map.get(x,0) == ts)
        to_keep.append('AJCC_'+str(ts))
    
        cdf['Pathological Grade_' + str(ts)] = pdata['Pathological Grade'].apply(lambda x: path_grade_map.get(x,0) == ts)
        to_keep.append('Pathological Grade_'+str(ts))
    subsites = [
            'BOT','GPS','Tonsil','Soft palate','NOS'
        #     'Pharyngeal wall'
    ]
    for subsite in subsites:
        cdf['subsite_'+subsite] = pdata['Tumor subsite (BOT/Tonsil/Soft Palate/Pharyngeal wall/GPS/NOS)'].apply(lambda x: x == subsite.replace(' ','_'))
        to_keep.append('subsite_'+subsite)
        
    for race in ['White/Caucasion', 'Hispanic/Latino', 'African American/Black', 'Asian']:
        cdf[race] = pdata['Race'].apply(lambda x: x==race)
        to_keep.append(race)
    return cdf[to_keep].set_index('id')

data.processed_df[get_camprt_data().columns]

,age,gender,bilateral,total_dose,dose_fraction,hpv,packs_per_year,T-category_1,N-category_0,AJCC_1,...,Pathological Grade_4,subsite_BOT,subsite_GPS,subsite_Tonsil,subsite_Soft palate,subsite_NOS,White/Caucasion,Hispanic/Latino,African American/Black,Asian
id,,,,,,,,,,,,,,,,,,,,,
3,55.969444,1,False,66.00,2.200000,1,0.0,0,0,1,...,0,1,0,0,0,0,True,False,False,False
5,20.950000,1,False,72.00,1.800000,0,38.0,0,0,0,...,0,1,0,0,0,0,True,False,False,False
6,69.930556,0,True,70.00,2.121212,1,35.0,0,0,0,...,0,1,0,0,0,0,True,False,False,False
7,72.319444,1,False,70.00,2.121212,1,0.0,1,0,0,...,0,0,0,0,0,1,True,False,False,False
8,59.730556,1,False,66.00,2.200000,1,0.0,1,0,1,...,0,0,0,1,0,0,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10201,49.566667,1,False,70.00,2.121212,1,30.0,0,0,0,...,0,1,0,0,0,0,True,False,False,False
10202,48.705556,1,False,72.00,1.714286,0,30.0,0,0,0,...,0,0,0,0,0,1,False,True,False,False
10203,77.116667,1,False,70.00,2.333333,1,0.0,1,0,1,...,0,0,0,1,0,0,True,False,False,False


In [14]:
lns = pd.read_csv('../data/monograms_clean.csv').drop(['RPLN_contra','RPLN_ipsi','id.1'],axis=1)
lns

,id,1A_ipsi,1B_ipsi,2A_ipsi,2B_ipsi,3_ipsi,4_ipsi,5A_ipsi,5B_ipsi,6_ipsi,1A_contra,1B_contra,2A_contra,2B_contra,3_contra,4_contra,5A_contra,5B_contra,6_contra
0,1,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588,10200,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
589,10202,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
590,10203,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
591,10204,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
import json
with open('../data/camprt_datajson.json','r') as f:
    camprtjson = json.load(f)
camprtjson
camprtdata = []
organs = set()
for item in camprtjson:
    entry = {'id': item['ID_int']}
    for organ, values in item['organData'].items():
        v = float(values['meanDose'])
        if np.isnan(v) or v == np.inf or v < 0 or v > 90:
            print('bad value',v)
            v = 0
        entry[organ] = v
        organs.add(organ)
    camprtdata.append(entry)
camprtdf = pd.DataFrame(camprtdata).set_index('id')
merged_camprt = camprtdf.merge(lns,on='id',how='inner').merge(get_camprt_data(),on='id').set_index('id')
organs = sorted(organs)
merged_camprt.fillna(0)

bad value 90.02


,Esophagus,Spinal_Cord,Lt_Brachial_Plexus,Rt_Brachial_Plexus,Cricopharyngeal_Muscle,Lt_thyroid_lobe,Rt_thyroid_lobe,Cricoid_cartilage,IPC,MPC,...,Pathological Grade_4,subsite_BOT,subsite_GPS,subsite_Tonsil,subsite_Soft palate,subsite_NOS,White/Caucasion,Hispanic/Latino,African American/Black,Asian
id,,,,,,,,,,,,,,,,,,,,,
4,40.64,31.63,48.80,69.54,8.46,48.10,57.91,7.92,51.68,63.35,...,False,True,False,False,False,False,True,False,False,False
11,39.22,31.94,56.92,48.21,29.38,57.14,51.76,34.06,51.92,71.09,...,False,True,False,False,False,False,True,False,False,False
27,1.17,14.50,1.36,45.87,4.01,1.02,44.56,6.76,6.91,26.24,...,False,False,False,True,False,False,True,False,False,False
29,30.73,26.28,44.65,52.98,27.08,49.57,55.30,27.40,19.49,57.43,...,False,True,False,False,False,False,False,True,False,False
33,32.89,25.98,55.89,53.36,41.72,61.80,54.90,45.25,44.32,46.52,...,False,False,False,True,False,False,True,False,False,False
34,36.35,27.25,44.37,53.07,25.86,50.42,56.14,25.85,13.76,46.39,...,False,False,False,True,False,False,True,False,False,False
35,25.54,26.51,49.40,46.52,13.07,44.35,50.04,17.47,20.97,63.86,...,False,True,False,False,False,False,True,False,False,False
36,30.80,28.35,51.05,48.22,9.26,46.78,50.68,15.14,41.35,58.76,...,False,True,False,False,False,False,True,False,False,False
41,32.43,26.50,42.35,55.48,14.13,47.02,54.45,20.40,56.38,65.58,...,False,True,False,False,False,False,True,False,False,False


In [54]:
def dose_mse_loss(ypred, y):
    #ignores loss in the autoencoder for missing values (-1 here)
    scale=y.shape[1]
    y = torch.flatten(y)
    y = y
    ypred = torch.flatten(ypred)
    out = ((ypred - y))**2
    loss = out.mean()/scale
    return loss

patience=100
epochs=10000
dose_save_file = '../resources/doseImputerTemp.pt'
lr = .1

ydf = merged_camprt[organs].fillna(0)
xdf = merged_camprt.drop(ydf.columns,axis=1).fillna(0)

ymax = ydf.max().max()
dosemodel = SymptomPredictor(xdf.shape[1],ydf.shape[1],
                             hidden_layers=[1000,100],
                             max_rating=ymax,dropout=.1)
train_ids = xdf.sample(frac=.7,replace=False).index
test_ids = xdf.drop(train_ids).index

xtrain = df_to_torch(xdf.loc[train_ids])
ytrain = df_to_torch(ydf.loc[train_ids])
xtest = df_to_torch(xdf.loc[test_ids])
ytest = df_to_torch(ydf.loc[test_ids])

dosemodel.fit_normalizer(xtrain)
optimizer = torch.optim.Adam(dosemodel.parameters(),lr=lr)
best_loss = 1000000000000000000000000
steps_since_improvement=0

dosemodel.train()
for epoch in range(epochs):
    optimizer.zero_grad()
    ypred = dosemodel(xtrain)
    loss = dose_mse_loss(ypred,ytrain)
    loss.backward()
    optimizer.step()
    ypred_test = dosemodel(xtest)
    val_loss = dose_mse_loss(ypred_test,ytest)
    print(epoch,loss.item(),val_loss.item())
    steps_since_improvement+=1
    if val_loss.item() < best_loss:
        best_loss = val_loss.item()
        steps_since_improvement=0
        torch.save(dosemodel.state_dict(),dose_save_file)
    if steps_since_improvement > patience:
        break
dosemodel.load_state_dict(torch.load(dose_save_file))
dosemodel.eval()
print('best loss',best_loss,epoch-patience)

0 43.059837341308594 42.487918853759766
1 41.928707122802734 38.931026458740234
2 38.32482147216797 34.30946731567383
3 33.42279815673828 30.164064407348633
4 28.985782623291016 27.12162971496582
5 25.46324348449707 25.67548942565918
6 23.962848663330078 25.157995223999023
7 23.93639373779297 24.741668701171875
8 24.153457641601562 24.48395347595215
9 23.970123291015625 23.863065719604492
10 23.16248893737793 23.440305709838867
11 22.474008560180664 22.895872116088867
12 22.15484046936035 22.608478546142578
13 21.907764434814453 22.235374450683594
14 21.63998031616211 21.859718322753906
15 21.364011764526367 21.50958824157715
16 21.08566665649414 21.299558639526367
17 20.83793830871582 20.97184944152832
18 20.414793014526367 20.58218002319336
19 19.981769561767578 20.08932113647461
20 19.486967086791992 19.554922103881836
21 18.928707122802734 18.62929344177246
22 18.151662826538086 18.198280334472656
23 17.443147659301758 17.380544662475586
24 16.74286460876465 16.464487075805664
25 1

In [55]:
torch.save(dosemodel,Const.dosemodel_file )
torch.load(Const.dosemodel_file)

SymptomPredictor(
  (layers): ModuleList(
    (0): Linear(in_features=50, out_features=1000, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1000, out_features=100, bias=True)
    (3): ReLU()
  )
  (batchnorm): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (relu): ReLU()
  (sigmoid): Sigmoid()
  (final_layer): Linear(in_features=100, out_features=47, bias=True)
)

In [81]:
def get_dose_knn(fdict,dmodel,camprt_df, organ_order=None,k=10):
    if organ_order is None:
        organ_order = Const.organ_order[:]
    
    camprt_input = camprt_df.drop(organ_order,axis=1)
    order = camprt_input.columns
    xin = torch.tensor([fdict[k] for k in order]).type(torch.FloatTensor).view(1,-1) 
    embeddings = dmodel.get_embedding(xin).cpu().detach().numpy()
    
    xalt = df_to_torch(camprt_input.fillna(0))
    alt_embeddings = dmodel.get_embedding(xalt).cpu().detach().numpy()
    
    dists = cdist(embeddings,alt_embeddings)[0]
    dorder = np.argsort(dists)[:k]
    dists = dists[dorder]
    
    camprt_doses = camprt_df[organ_order]
    sim_doses = camprt_doses.iloc[dorder]
    mean_dose = sim_doses.mean().to_dict()
    result = {}
    for organ,meanval in mean_dose.items():
        entry = {'mean': meanval}
        other = sim_doses[organ].values.tolist()
        entry['max'] = np.max(other)
        entry['min'] = np.min(other)
        entry['values'] = other
        result[organ] = entry
    dists = [np.round(d,2) for d in dists]
    return {'ids': camprt_df.index[dorder].tolist(), 'distances':dists, 'doses': result}

get_dose_knn(test_patient,dosemodel,merged_camprt)

{'ids': [10164, 10147, 10092, 218, 10135, 10044, 262, 10129, 179, 10138],
 'distances': [1.84, 2.03, 2.77, 3.06, 3.18, 3.7, 3.78, 3.81, 3.87, 3.91],
 'doses': {'Esophagus': {'mean': 34.071,
   'max': 37.95,
   'min': 25.29,
   'values': [34.98,
    35.31,
    33.97,
    34.9,
    33.36,
    32.64,
    25.29,
    35.05,
    37.95,
    37.26]},
  'Spinal_Cord': {'mean': 28.15,
   'max': 35.52,
   'min': 24.31,
   'values': [35.52,
    27.42,
    24.31,
    25.99,
    27.49,
    27.44,
    27.84,
    31.63,
    28.65,
    25.21]},
  'Lt_Brachial_Plexus': {'mean': 49.853,
   'max': 57.92,
   'min': 38.66,
   'values': [49.16,
    53.97,
    46.64,
    38.66,
    48.94,
    48.41,
    49.7,
    54.08,
    57.92,
    51.05]},
  'Rt_Brachial_Plexus': {'mean': 54.21,
   'max': 60.86,
   'min': 45.71,
   'values': [58.78,
    57.47,
    54.9,
    55.49,
    58.28,
    57.11,
    60.86,
    46.31,
    47.19,
    45.71]},
  'Cricopharyngeal_Muscle': {'mean': 18.646,
   'max': 35.96,
   'min': 6.6

In [105]:
def get_dose_prediction(fdict,dmodel,camprt_df, organ_order=None):
    if organ_order is None:
        organ_order = Const.organ_order[:]
    
    camprt_input = camprt_df.drop(organ_order,axis=1)
    order = camprt_input.columns
    xin = torch.tensor([fdict[k] for k in order]).type(torch.FloatTensor).view(1,-1) 
    ypred = dmodel(xin).cpu().detach().numpy()[0]
    
    return {o:np.round(y,2 for o,y in zip(organ_order, ypred)}
get_dose_prediction(test_patient,dosemodel,merged_camprt)

{'Esophagus': 15.907598,
 'Spinal_Cord': 31.586063,
 'Lt_Brachial_Plexus': 23.10606,
 'Rt_Brachial_Plexus': 31.531345,
 'Cricopharyngeal_Muscle': 53.54892,
 'Lt_thyroid_lobe': 86.25,
 'Rt_thyroid_lobe': 55.870605,
 'Cricoid_cartilage': 64.25154,
 'IPC': 36.65653,
 'MPC': 66.13756,
 'Brainstem': 35.4671,
 'Larynx': 27.172567,
 'Thyroid_cartilage': 26.4124,
 'Rt_Sternocleidomastoid_M': 56.649967,
 'Rt_Mastoid': 0.0,
 'Rt_Parotid_Gland': 46.541172,
 'Rt_Medial_Pterygoid_M': 39.9338,
 'Rt_Lateral_Pterygoid_M': 30.849422,
 'Rt_Masseter_M': 40.807407,
 'Lt_Sternocleidomastoid_M': 55.16181,
 'Lt_Mastoid': 31.036226,
 'Lt_Parotid_Gland': 0.0,
 'Lt_Submandibular_Gland': 56.86295,
 'Lt_Medial_Pterygoid_M': 59.881886,
 'Lt_Lateral_Pterygoid_M': 45.63928,
 'Lt_Masseter_M': 62.5084,
 'Supraglottic_Larynx': 42.579956,
 'SPC': 60.710022,
 'Rt_Submandibular_Gland': 58.45572,
 'Hyoid_bone': 0.0,
 'Soft_Palate': 53.09308,
 'Genioglossus_M': 46.11689,
 'Tongue': 37.72832,
 'Rt_Ant_Digastric_M': 48.81045,

In [ ]:
mdasi_input